In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import string
import contractions
import nltk
from nltk.corpus import stopwords
import torch
import transformers
from transformers import AdamW, ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification, ElectraModel, AutoTokenizer, TrainingArguments, DataCollatorWithPadding, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
import evaluate
from torch.optim import AdamW
from finetuning_scheduler import FinetuningScheduler
from pytorch_lightning import Trainer, Callback
from pytorch_lightning.callbacks import LearningRateMonitor, ModelSummary, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torchmetrics
from torchmetrics import Accuracy, Precision, Recall
from torchmetrics.classification import BinaryF1Score
import tensorboard
import matplotlib.pyplot as plt
import os
from torch.optim.lr_scheduler import OneCycleLR, LambdaLR, CosineAnnealingLR
import json
from sklearn.model_selection import train_test_split
import math


/home/vscode/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext tensorboard



In [2]:
rm -rf ./tb_logs/


In [4]:
%reload_ext tensorboard

In [5]:
data_path = "/workspaces/sarcasm_detection/notebooks/project_data/Sarcasm_Headlines_Dataset_v2.json"
#tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
# model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels = 2)
#configuration = ElectraConfig()
#model = ElectraForSequenceClassification(configuration)
version_number = 1
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_path = f"sarcasm_detection_finetune_ckpt_v{version_number}_{current_time}.ckpt"

In [6]:
#data module
class SarcasmDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
            self.data = data
            self.tokenizer = tokenizer
            self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]['headline']
        labels = self.data[idx]['is_sarcastic']
        encodings = self.tokenizer(
              text, 
              max_length=self.max_length, 
              padding='max_length', 
              truncation=True, 
              return_attention_mask=True, 
              return_token_type_ids=False, 
              return_tensors='pt'
              )
        return encodings['input_ids'].flatten(), encodings['attention_mask'].flatten(), torch.tensor(labels)
    
class SarcasmDataModule(pl.LightningDataModule):
    def __init__(self, data_path, batch_size, tokenizer="google/electra-small-discriminator"):
        super().__init__()
        self.data_path = data_path
        self.tokenizer = ElectraTokenizer.from_pretrained(tokenizer)
        self.batch_size=batch_size
        self.collate_fn = self.default_collate_fn

    
    def prepare_data(self):
        col_types = {'headline':'str', 'is_sarcastic':'int32'}

        df = (
             pd.read_json(self.data_path, lines=True)
             .drop(columns=['article_link'])       
             .astype(col_types)
        )

        train_df, val_df, test_df = self.split_datasets(df)
        # print(f"training df length: {len(train_df)}")
        # print(f"Validation DataFrame length: {len(val_df)}")
        # print(f"Test DataFrame length: {len(test_df)}")
        # print(f"total df len: {len(train_df+val_df+test_df)}")
        # print(len(df))
        
        self.data_train = train_df.to_dict('records')
        self.data_val = val_df.to_dict('records')
        self.data_test = test_df.to_dict('records')
        
    def setup(self, stage: str=None):

        if stage == "fit":
            self.train_dataset = SarcasmDataset(self.data_train, self.tokenizer)
            self.val_dataset = SarcasmDataset(self.data_val, self.tokenizer)
        
        if stage == "test":
            self.test_dataset = SarcasmDataset(self.data_test, self.tokenizer)
        
        if stage == "predict":
            self.predict_dataset = SarcasmDataset(self.data_test, self.tokenizer)

    def steps_per_epoch(self):
        return len(self.train_dataset)
    
    def split_datasets(self, df):
        train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
        val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)
        return train_df, val_df, test_df

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=6, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=6, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=6, shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, batch_size=self.batch_size, num_workers=6, shuffle=False, collate_fn=self.collate_fn)
    
    def default_collate_fn(self, batch):
        input_ids, attention_mask, labels = zip(*batch)
        input_ids = torch.stack(input_ids)
        attention_mask = torch.stack(attention_mask)
        labels = torch.stack(labels)
        return input_ids, attention_mask, labels


In [15]:


class ElectraClassifier(pl.LightningModule):
    def __init__(self, data_module, batch_size, model_name="google/electra-small-discriminator", num_labels=2, learning_rate=2e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.data_module = data_module
        self.num_layers = len(list(self.parameters()))
        self.warmup_steps = 10000
        #self.finetuning_scheduler = FinetuningScheduler()
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.predictions = []
        self.targets = []

        for param in self.model.electra.parameters():
            param.requires_grad = False

        #metrics
        self.train_accuracy = Accuracy(task='binary', num_classes=num_labels)
        self.val_accuracy = Accuracy(task='binary', num_classes=num_labels)
        self.train_precision = Precision(task='binary', num_classes=num_labels, average = 'weighted')
        self.val_precision = Precision(task='binary', num_classes=num_labels, average = 'weighted')
        self.train_recall = Recall(task='binary', num_classes=num_labels, average = 'weighted')
        self.val_recall = Recall(task='binary', num_classes=num_labels, average = 'weighted')
        self.val_f1_score = BinaryF1Score(task='binary', num_classes=num_labels)
        self.f1 = BinaryF1Score(task='binary', num_classes=num_labels, average = 'macro')

        #self.finetuning_scheduler.freeze(self.model.electra)

        #for adding smaller networks on top
        #self.dropout = nn.Dropout(0.1)
        #self.fc1 = nn.Linear(self.electra.config.hidden_size, 128)
        #self.fc2 = nn.Linear(128, num_labels)

    def train_dataloader(self):
        return self.data_module.train_dataloader()

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    
        #for adding smaller networks on top
        #outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        #last_hidden_state = outputs.last_hidden_state
        #cls_token = last_hidden_state[:, 0]
        #x = self.dropout(cls_token)
        #x = self.fc1(x)
        #x = torch.relu(x)
        #x = self.fc2(x)
        #return x
    
    def on_train_start(self):
        self.train_dataset_len = len(self.train_dataloader().dataset)
    
    def on_train_batch_start(self, batch, batch_idx):

        if self.global_step == self.warmup_steps:
            #unfreeze base layers
            print("unfrozen")
            for param in self.model.electra.parameters():
                param.requires_grad = True

            steps_per_epoch = len(self.trainer.datamodule.train_dataloader()) // self.hparams.batch_size

        
        #print("Current learning rate at global step {}: {}".format(self.global_step, self.current_lr))

    def on_test_epoch_end(self, test_step_outputs):
        all_preds = torch.cat([x['preds'] for x in test_step_outputs])
        all_labels = torch.cat([x['labels'] for x in test_step_outputs])
        return {'preds': all_preds, 'labels': all_labels}

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        preds = outputs.logits.argmax(dim=-1)

        #logging
        acc = self.train_accuracy(preds, labels)
        prec = self.train_precision(preds, labels)
        rec = self.train_recall(preds, labels)
        self.log("train_loss", loss, on_step = True, on_epoch = True, prog_bar = True)
        self.log("train_accuracy", acc, on_step = True, on_epoch = True, prog_bar=True)
        self.log("train_precision", prec, on_step = True, on_epoch = True, prog_bar=True)
        self.log("train_recall", rec, on_step = True, on_epoch = True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        preds = outputs.logits.argmax(dim=-1)

        #logging
        acc = self.val_accuracy(preds, labels)
        prec = self.val_precision(preds, labels)
        rec = self.val_recall(preds, labels)
        self.log("val_loss", loss)
        self.log("val_accuracy", acc)
        self.log("val_precision", prec)
        self.log("val_recall", rec)

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        preds = outputs.logits.argmax(dim=-1)
        f1_score = self.f1(preds, labels)
        self.log('test_f1', f1_score, on_step=True, on_epoch=True, prog_bar=True)
        return {'preds': preds, 'labels': labels}
    
    def predict_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        preds = outputs.logits.argmax(dim=-1)
        return {'preds':preds, 'labels':labels}
    
    def collate_fn(self, batch):
        inputs, labels = zip(*batch)
        return torch.stack(inputs), torch.stack(labels)

    @property
    def current_lr(self):
        return self.optimizers().param_groups[0]['lr']
    
    def configure_optimizers(self):

        optimizer = AdamW(self.parameters(), lr=self.hparams.learning_rate)
    
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.warmup_steps,
            num_training_steps=self.trainer.estimated_stepping_batches
        )

        scheduler = {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1
        }
        return [optimizer], [scheduler]

    # def configure_optimizers(self):

    #     num_epochs = self.trainer.max_epochs

    #     if self.trainer.state.stage == 'fit':
    #         #steps_per_epoch = len(self.train_dataloader()) // self.hparams.batch_size
    #         steps_per_epoch = len(self.trainer.datamodule.train_dataloader())
    #     else:
    #         steps_per_epoch = 1
        
    #     total_steps = num_epochs * steps_per_epoch

    #     optimizer = AdamW(self.parameters(), lr=self.hparams.learning_rate)
    #     scheduler = {
    #         'scheduler': OneCycleLR(
    #             optimizer,
    #             max_lr=self.hparams.learning_rate,
    #             total_steps=total_steps,
    #             anneal_strategy='linear'
    #         ),
    #         'name':'learning_rate',
    #         'interval':'step',
    #         'frequency': 1
    #     }
    #     return [optimizer],[scheduler]



In [ ]:
class ElectraSubClassification(pl.LightningModule):
    

In [8]:
#metrics plotting

class MetricsCallback(Callback):
    def __init__(self):
        super().__init__()
        self.train_metrics = []
        self.val_metrics = []
        self.batch_train_metrics = []
    
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        self.batch_train_metrics.append(trainer.callback_metrics)

    def on_train_epoch_end(self, trainer, pl_module):
        epoch_train_metrics = {}
        for key in self.batch_train_metrics[0].keys():
            epoch_train_metrics[key] = torch.stack([x[key] for x in self.batch_train_metrics]).mean()
        self.train_metrics.append(epoch_train_metrics)
        self.batch_train_metrics = []

    def on_validation_epoch_end(self, trainer, pl_module):
        self.val_metrics.append(trainer.callback_metrics)

# class CustomFineTuningCallback(Callback):
#         def __init__(self, num_layers, num_layers_to_unfreeze, warmup_steps):
#              self.num_layers = num_layers
#              self.num_layers_to_unfreeze = num_layers_to_unfreeze
#              self.warmup_steps = warmup_steps
#              self.optimizer = None
        
#         def on_train_start(self, trainer, pl_module):
#             self.optimizer = trainer.optimizers[0]


#         def on_train_batch_start(self, trainer, pl_module, batch, batch_idx):
#             global_step = trainer.global_step

#             if global_step >= self.warmup_steps:
#                   steps_since_warmup = global_step - self.warmup_steps

#                   if steps_since_warmup % self.num_layers_to_unfreeze == 0:
#                     layer_to_unfreeze = self.num_layers - (steps_since_warmup // self.num_layers_to_unfreeze)
#                   for i, param_group in enumerate(trainer.optimizers[0].param_groups):
#                     param_group["requires_grad"] = i >= layer_to_unfreeze
            
#             trainer.callbacks[1].on_train_batch_start(trainer, pl_module, batch, batch_idx)

In [9]:
data_module = SarcasmDataModule(data_path=data_path, batch_size=16)
model = ElectraClassifier(data_module = data_module, batch_size=data_module.batch_size)
lr_monitor = LearningRateMonitor(logging_interval = 'step')
logger = TensorBoardLogger("tb_logs", name="electra_model_v1")
metrics_callback = MetricsCallback()
early_stopping = EarlyStopping('val_loss', patience=5, verbose=True)

trainer = Trainer(
    max_epochs=10000,
    callbacks=[
        lr_monitor, 
        metrics_callback, 
        early_stopping,
    ],
    logger=logger
)



Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [10]:


trainer.fit(model, data_module)

trainer.save_checkpoint(checkpoint_path)

Missing logger folder: tb_logs/electra_model_v1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type                             | Params
---------------------------------------------------------------------
0 | model           | ElectraForSequenceClassification | 13.5 M
1 | train_accuracy  | BinaryAccuracy                   | 0     
2 | val_accuracy    | BinaryAccuracy                   | 0     
3 | train_precision | BinaryPrecision                  | 0     
4 | val_precision   | BinaryPrecision                  | 0     
5 | train_recall    | BinaryRecall                     | 0     
6 | val_recall      | BinaryRecall                     | 0     
7 | val_f1_score    | BinaryF1Score                    | 0     
8 | f1              | BinaryF1Score                    | 0     
---------------------------------------------------------------------
66.3 K    Trainable params
13.5 M    Non-trainable params
13.5

Epoch 0: 100%|██████████| 1431/1431 [01:33<00:00, 15.27it/s, v_num=0, train_loss_step=0.666, train_accuracy_step=0.600, train_precision_step=0.889, train_recall_step=0.615, train_loss_epoch=0.689, train_accuracy_epoch=0.546, train_precision_epoch=0.535, train_recall_epoch=0.796]

Metric val_loss improved. New best score: 0.670


Epoch 1: 100%|██████████| 1431/1431 [01:35<00:00, 15.01it/s, v_num=0, train_loss_step=0.664, train_accuracy_step=0.533, train_precision_step=0.750, train_recall_step=0.333, train_loss_epoch=0.653, train_accuracy_epoch=0.672, train_precision_epoch=0.724, train_recall_epoch=0.510]

Metric val_loss improved by 0.057 >= min_delta = 0.0. New best score: 0.613


...

Epoch 8: 100%|██████████| 1431/1431 [03:21<00:00,  7.11it/s, v_num=0, train_loss_step=0.273, train_accuracy_step=0.867, train_precision_step=0.750, train_recall_step=1.000, train_loss_epoch=0.203, train_accuracy_epoch=0.917, train_precision_epoch=0.913, train_recall_epoch=0.916]  

Metric val_loss improved by 0.039 >= min_delta = 0.0. New best score: 0.210


Epoch 13: 100%|██████████| 1431/1431 [03:21<00:00,  7.09it/s, v_num=0, train_loss_step=0.00204, train_accuracy_step=1.000, train_precision_step=1.000, train_recall_step=1.000, train_loss_epoch=0.0439, train_accuracy_epoch=0.984, train_precision_epoch=0.984, train_recall_epoch=0.984] 

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.210. Signaling Trainer to stop.


Epoch 13: 100%|██████████| 1431/1431 [03:22<00:00,  7.07it/s, v_num=0, train_loss_step=0.00204, train_accuracy_step=1.000, train_precision_step=1.000, train_recall_step=1.000, train_loss_epoch=0.0439, train_accuracy_epoch=0.984, train_precision_epoch=0.984, train_recall_epoch=0.984]


In [12]:
%tensorboard --logdir=tb_logs/

In [17]:
def find_latest_checkpoint(version_prefix = "sarcasm_detection_finetune_ckpt_v"):
    checkpoints = [file for file in os.listdir() if file.startswith(version_prefix) and file.endswith(".ckpt")]
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

In [19]:
latest_checkpoint = find_latest_checkpoint()

if latest_checkpoint:

    #load model
    loaded_model = ElectraClassifier.load_from_checkpoint(latest_checkpoint)

    #new model with different classification heads
    transfer_model = ElectraClassifier(data_module=data_module, num_labels=6, batch_size=data_module.batch_size)

    #load fine_tuned model weights keeping new classification head
    transfer_model.electra.load_state_dict(loaded_model.electra.state_dict())

else:
    print("No model checkpoint found")

/home/vscode/.local/lib/python3.9/site-packages/lightning/pytorch/_graveyard/legacy_import_unpickler.py:23: UserWarning: Redirecting import of pytorch_lightning.trainer.trainer.Trainer to lightning.pytorch.trainer.trainer.Trainer
  warnings.warn(f"Redirecting import of {module}.{name} to {new_module}.{name}")
/home/vscode/.local/lib/python3.9/site-packages/lightning/pytorch/_graveyard/legacy_import_unpickler.py:23: UserWarning: Redirecting import of pytorch_lightning.trainer.states.TrainerState to lightning.pytorch.trainer.states.TrainerState
  warnings.warn(f"Redirecting import of {module}.{name} to {new_module}.{name}")
/home/vscode/.local/lib/python3.9/site-packages/lightning/pytorch/_graveyard/legacy_import_unpickler.py:23: UserWarning: Redirecting import of pytorch_lightning.trainer.states.TrainerStatus to lightning.pytorch.trainer.states.TrainerStatus
  warnings.warn(f"Redirecting import of {module}.{name} to {new_module}.{name}")
/home/vscode/.local/lib/python3.9/site-packages/l

AttributeError: 'ElectraClassifier' object has no attribute 'electra'

In [16]:
#testing

test_result = trainer.test(model, data_module)
test_preds = test_result[0]['preds']
test_labels = test_result[0]['labels']
#print(f"test f1 score: {test_f1_score}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [03:14, ?it/s]


AttributeError: 'ElectraClassifier' object has no attribute 'predictions'

In [33]:
predict_result = trainer.predict(model, data_module)
predict_preds = torch.cat([torch.tensor(x['preds']) for x in predict_result])
predict_labels = torch.cat([torch.tensor(x['labels'] for x in predict_result)])
predict_f1_score = model.f1(predict_preds, predict_labels).item()
print(f"predict f1 score: {predict_f1_score}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


AttributeError: 'SarcasmDataModule' object has no attribute 'collate_fn'

In [19]:
#find most recent version of model checkpoint

def find_latest_checkpoint(version_prefix = "sarcasm_detection_finetune_ckpt_v"):
    checkpoints = [file for file in os.listdir() if file.startswith(version_prefix) and file.endswith(".ckpt")]
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

In [20]:
latest_checkpoint = find_latest_checkpoint()

if latest_checkpoint:

    #load model
    loaded_model = ElectraClassifier.load_from_checkpoint(latest_checkpoint)

    #new model with different classification heads
    transfer_model = ElectraClassifier(num_labels=6)

    #load fine_tuned model weights keeping new classification head
    transfer_model.electra.load_state_dict(loaded_model.electra.state_dict())

else:
    print("No model checkpoint found")

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

AttributeError: 'ElectraClassifier' object has no attribute 'electra'

In [31]:
model.eval()

ElectraClassifier(
  (model): ElectraForSequenceClassification(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0-11): 12 x ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=256, out_features=256, bias=True)
                (key): Linear(in_features=256, out_features=256, bias=True)
                (value): Linear(in_features=256, out_features=256, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              

In [ ]:
# Create a new model with a different number of classes
transfer_model = ElectraClassifier(num_labels=3)

# Load the fine-tuned model weights, but keep the new classification head
transfer_model.model.electra.load_state_dict(loaded_model.model.electra.state_dict())

In [ ]:
#evaluation

predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)

In [ ]:
metric = evaluate.load("f1", "accuracy", "precision")
results = metric.compute(predictions=preds, references=predictions.label_ids)

In [83]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy", "precision", "f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions = predictions, references = labels)

In [71]:


trainer = Trainer(
    model, 
    training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
#evaluation

predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)

In [ ]:
metric = evaluate.load("f1", "accuracy", "precision")
results = metric.compute(predictions=preds, references=predictions.label_ids)


In [ ]:

# train_encodings = tokenizer.batch_encode_plus(list(train_df['headline']), max_length=512, padding='max_length', truncation=True, return_attention_mask=True, return_token_type_ids=False, return_tensors='pt')
# val_encodings = tokenizer.batch_encode_plus(list(val_df['headline']), max_length=512, padding='max_length', truncation=True, return_attention_mask=True, return_token_type_ids=False, return_tensors='pt')
# test_encodings = tokenizer.batch_encode_plus(list(test_df['headline']), max_length=512, padding=True, truncation=True, return_attention_mask=True, return_token_type_ids=False, return_tensors='pt')

# train_input_ids = torch.tensor(train_encodings['input_ids'])
# train_attention_masks = torch.tensor(train_encodings['attention_mask'])
# val_input_ids = torch.tensor(val_encodings['input_ids'])
# val_attention_masks = torch.tensor(val_encodings['attention_mask'])
# test_input_ids = torch.tensor(test_encodings['input_ids'])
# test_attention_masks = torch.tensor(test_encodings['attention_mask'])

# train_labels = torch.tensor(train_df['is_sarcastic'].values)
# val_labels = torch.tensor(val_df['is_sarcastic'].values)
# test_labels = torch.tensor(test_df['is_sarcastic'].values)
     
        # steps_per_epoch = len(self.trainer.datamodule.train_dataloader()) // self.hparams.batch_size
        # T_0 = max(steps_per_epoch // 2, 1)
        # custom_scheduler = CustomScheduler(optimizer, self.warmup_steps, T_0)

        # return {
        # 'optimizer': optimizer,
        # 'lr_scheduler': {
        #     'scheduler': custom_scheduler,
        #     'interval': 'step',
        #     'frequency': 1
        #     }
        # }

        
        # warmup_steps = 10000
        # total_steps = num_epochs * steps_per_epoch
        # num_epochs = self.trainer.max_epochs

        # if self.trainer.state.stage == 'fit':
        #     steps_per_epoch = len(self.train_dataloader()// self.hparams.batch_size)
        # else:
        #     steps_per_epoch = 1

        # warmup_steps = 10000
        # total_steps = num_epochs * steps_per_epoch

        # if total_steps <= warmup_steps:
        #     total_steps = warmup_steps+1

        # warmup_scheduler = get_cosine_schedule_with_warmup(
        #     optimizer,
        #     num_warmup_steps=warmup_steps,
        #     num_training_steps=total_steps,
        # )

        # T_0 = max(steps_per_epoch // 2, 1)
        # T_mult = 1
        # adaptive_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0, T_mult)

        # def combined_scheduler(step):
        #     if step < warmup_steps:
        #         return warmup_scheduler.get_lr()[0]
        #     else: 
        #         return adaptive_scheduler.get_lr()[0]

        # scheduler = LambdaLR(optimizer, combined_scheduler)

        # scheduler_config = {
        #     'scheduler':scheduler,
        #     'name':'learning_rate',
        #     'interval':'step',
        #     'frequency':1
        # }

        # # def lr_lambda(step):
        # #     if step < 10000:
        # #         return 0.00002
        # #     elif step < 20000:
        # #         return 0.0002
        # #     else:
        # #         return 0.002
            
        
        # # lr_scheduler = {
        # #     'scheduler':LambdaLR(optimizer, lr_lambda),
        # #     'name': 'learning_rate',
        # #     'interval':'step',
        # #     'frequency': 1,

        # # }

        # print('initial learning rate:', optimizer.param_groups[0]['lr'])
    
        # return [optimizer], [scheduler]

    #     optimizer = AdamW(self.parameters(), lr=self.hparams.learning_rate)
    #     scheduler_config = {
    #     'scheduler': FinetuningScheduler(
    #         optimizer,
    #         milestones=[
    #             (0, 0.00002),
    #             (10000, 0.0002),
    #             (20000, 0.002)
    #         ],
    #     ),
    #     'name': 'learning_rate',
    #     'interval': 'step',
    #     'frequency': 1
    # }
    #     return [optimizer],[scheduler_config]

# class CustomScheduler:
#     def __init__(self, optimizer, warmup_steps, T_0, T_mult = 1):
#         self.optimizer = self.optimizer
#         self.warmup_steps = warmup_steps
#         self.scheduler = CosineAnnealingLR(optimizer, T_0, T_mult)


#     def step(self, global_step):
#         if global_step > self.warmup_steps:
#             self.scheduler.step(global_step - self.warmup_steps)

#             for i, param_group in enumerate(self.optimizer.param_groups):
#                 param_group['lr'] = self.scheduler.get_last_lr()[i]

# class CustomScheduler:
#     def __init__(self, optimizer, warmup_steps, T_0, T_mult=1):
#         self.optimizer = optimizer
#         self.warmup_steps = warmup_steps
#         self.scheduler = CosineAnnealingLR(optimizer, T_0, T_mult)

#     def step(self, global_step):
#         if global_step > self.warmup_steps:
#             self.scheduler.step(global_step - self.warmup_steps)

#             for i, param_group in enumerate(self.optimizer.param_groups):
#                 param_group['lr'] = self.scheduler.get_last_lr()[i]
    
#     def state_dict(self):
#         return self.scheduler.state_dict()

#     def load_state_dict(self, state_dict):
#         self.scheduler.load_state_dict(state_dict)


    
    # def on_train_batch_start(self, batch, batch_idx):

    #     if self.global_step < self.warmup_steps:
    #         for param in self.model.electra.parameters():
    #             param.requires_grad = False

    #     if self.global_step == self.warmup_steps:
    #         #unfreeze base layers
    #         for param in self.model.electra.parameters():
    #             param.requires_grad = True

    #         steps_per_epoch = len(self.trainer.datamodule.train_dataloader()) // self.hparams.batch_size
    #         #T_0 = max(steps_per_epoch // 2, 1)

    #         #self.custom_scheduler = CustomScheduler(self.optimizers, self.warmup_steps, T_0)

    #     # if self.global_step > self.warmup_steps:
    #     #     self.custom_scheduler.step(self.global_step)
        
    #     # current_lr = self.optimizers().param_groups[0]['lr']